# A/B 테스트 캠페인 퍼널 분석
## 📊 컨트롤 vs 테스트 캠페인 성과에 대한 통계 분석

### 분석 목표:
- 컨트롤 그룹과 테스트 그룹 간 캠페인 성과 비교
- 전환율에 대한 통계적 가설 검정
- 최적화 기회 식별을 위한 퍼널 분석
- 캠페인 전략을 위한 데이터 기반 권장사항

## 1. 데이터 준비 및 품질 평가

In [ ]:
# 포괄적인 데이터 분석을 위한 필수 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 통계 검정 라이브러리
from scipy import stats
from scipy.stats import ttest_ind, mannwhitneyu, chi2_contingency
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# 더 나은 시각화를 위한 스타일 설정
plt.style.use('seaborn')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

In [ ]:
def load_and_preprocess_data():
    """
    포괄적인 데이터 품질 검사를 통한 캠페인 데이터 로드 및 전처리
    """
    # 일관된 네이밍을 위한 컬럼 매핑
    column_names = ['campaign_name', 'date', 'spend', 'impression', 'reach', 
                   'click', 'search', 'view', 'cart', 'purchase']
    
    # 컨트롤 그룹 로드
    control_df = pd.read_csv("control_group.csv", sep=";")
    control_df['Date'] = pd.to_datetime(control_df['Date'], format='%d.%m.%Y')
    control_df.columns = column_names
    control_df['campaign_name'] = 'control'
    
    # 테스트 그룹 로드
    test_df = pd.read_csv("test_group.csv", sep=";")
    test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d.%m.%Y')
    test_df.columns = column_names
    test_df['campaign_name'] = 'test'
    
    return control_df, test_df

def data_quality_assessment(df, campaign_name):
    """
    포괄적인 데이터 품질 평가
    """
    print(f"\n=== {campaign_name.upper()} 캠페인 데이터 품질 ===\n")
    
    # 기본 정보
    print(f"📊 데이터셋 크기: {df.shape}")
    print(f"📅 날짜 범위: {df['date'].min().strftime('%Y-%m-%d')}부터 {df['date'].max().strftime('%Y-%m-%d')}까지")
    print(f"⏱️  기간: {(df['date'].max() - df['date'].min()).days + 1}일")
    
    # 결측 데이터 분석
    missing_data = df.isnull().sum()
    missing_percent = 100 * missing_data / len(df)
    
    if missing_data.sum() > 0:
        print("\n🚨 결측 데이터 발견:")
        for col in missing_data[missing_data > 0].index:
            print(f"   {col}: {missing_data[col]}개 행 ({missing_percent[col]:.1f}%)")
    else:
        print("\n✅ 결측 데이터 없음")
    
    # 주요 지표에 대한 통계 요약
    key_metrics = ['spend', 'impression', 'click', 'purchase']
    print("\n📈 주요 지표 요약:")
    summary_stats = df[key_metrics].describe()
    print(summary_stats.round(2))
    
    return {
        'missing_data': missing_data,
        'summary_stats': summary_stats,
        'date_range': (df['date'].min(), df['date'].max())
    }

# 데이터 로드
control_df, test_df = load_and_preprocess_data()

# 데이터 품질 평가
control_quality = data_quality_assessment(control_df, '컨트롤')
test_quality = data_quality_assessment(test_df, '테스트')

## 2. 결측 데이터 처리 및 민감도 분석
### 결론의 견고성 테스트를 위한 다중 시나리오

In [ ]:
def create_analysis_scenarios(control_df, test_df):
    """
    민감도 분석을 위한 다중 시나리오 생성
    """
    scenarios = {}
    
    # 숫자형 컬럼 식별
    numeric_cols = control_df.select_dtypes(include=[np.number]).columns
    
    # 시나리오 1: 결측 데이터가 있는 행 제거
    control_clean = control_df.dropna()
    test_clean = test_df.dropna()
    
    # 두 데이터셋에 모두 존재하는 날짜만 포함
    common_dates = set(control_clean['date']) & set(test_clean['date'])
    control_s1 = control_clean[control_clean['date'].isin(common_dates)]
    test_s1 = test_clean[test_clean['date'].isin(common_dates)]
    
    scenarios['complete_cases'] = pd.concat([control_s1, test_s1], ignore_index=True)
    
    # 시나리오 2: 중앙값으로 채우기 (이상치에 더 강건)
    control_median = control_df.copy()
    control_median[numeric_cols] = control_median[numeric_cols].fillna(control_median[numeric_cols].median())
    scenarios['median_fill'] = pd.concat([control_median, test_df], ignore_index=True)
    
    # 시나리오 3: 평균값으로 채우기
    control_mean = control_df.copy()
    control_mean[numeric_cols] = control_mean[numeric_cols].fillna(control_mean[numeric_cols].mean())
    scenarios['mean_fill'] = pd.concat([control_mean, test_df], ignore_index=True)
    
    return scenarios

# 분석 시나리오 생성
analysis_scenarios = create_analysis_scenarios(control_df, test_df)

print("📋 분석 시나리오 생성 완료:")
for name, df in analysis_scenarios.items():
    control_rows = len(df[df['campaign_name'] == 'control'])
    test_rows = len(df[df['campaign_name'] == 'test'])
    print(f"   {name}: 컨트롤={control_rows}, 테스트={test_rows}, 전체={len(df)}")

# 주 분석에는 가장 보수적인 시나리오(완전 사례) 사용
primary_df = analysis_scenarios['complete_cases'].copy()
print(f"\n✅ 주 분석에 사용할 시나리오: complete_cases")
print(f"📊 최종 데이터셋 크기: {primary_df.shape}")

## 3. 탐색적 데이터 분석 및 캠페인 개요

In [ ]:
def comprehensive_eda(df):
    """
    캠페인 성과에 대한 포괄적인 탐색적 데이터 분석
    """
    print("\n" + "="*60)
    print("           📊 캠페인 성과 개요")
    print("="*60)
    
    # 캠페인 레벨 집계 지표
    campaign_summary = df.groupby('campaign_name').agg({
        'spend': ['sum', 'mean', 'std'],
        'impression': ['sum', 'mean'],
        'click': ['sum', 'mean'],
        'search': ['sum', 'mean'],
        'view': ['sum', 'mean'],
        'cart': ['sum', 'mean'],
        'purchase': ['sum', 'mean']
    }).round(2)
    
    print("\n🎯 캠페인 요약 통계:")
    print(campaign_summary)
    
    # 주요 성과 지표 계산
    kpi_summary = df.groupby('campaign_name').apply(calculate_kpis).round(4)
    print("\n📈 주요 성과 지표:")
    print(kpi_summary)
    
    return campaign_summary, kpi_summary

def calculate_kpis(group_df):
    """
    캠페인 분석을 위한 포괄적인 KPI 계산
    """
    total_spend = group_df['spend'].sum()
    total_impressions = group_df['impression'].sum()
    total_clicks = group_df['click'].sum()
    total_purchases = group_df['purchase'].sum()
    total_views = group_df['view'].sum()
    total_carts = group_df['cart'].sum()
    
    return pd.Series({
        'CTR': (total_clicks / total_impressions) * 100,  # 클릭률 (%)
        'CPC': total_spend / total_clicks,  # 클릭당 비용
        'CPM': (total_spend / total_impressions) * 1000,  # 천 노출당 비용
        'CPA': total_spend / total_purchases,  # 구매당 비용
        'Conversion_Rate': (total_purchases / total_clicks) * 100,  # 구매 전환율 (%)
        'View_to_Cart': (total_carts / total_views) * 100,  # 조회-장바구니 전환율 (%)
        'Cart_to_Purchase': (total_purchases / total_carts) * 100,  # 장바구니-구매 전환율 (%)
        'ROAS_Proxy': total_purchases / total_spend,  # 지출 대비 구매수 (ROAS 대리지표)
    })

# 포괄적인 EDA 수행
campaign_summary, kpi_summary = comprehensive_eda(primary_df)

## 4. 통계적 가설 검정 프레임워크

In [ ]:
def statistical_testing_suite(df, alpha=0.05):
    """
    A/B 테스트를 위한 포괄적인 통계 검정 프레임워크
    """
    print("\n" + "="*60)
    print("           🔬 통계적 가설 검정")
    print("="*60)
    print(f"유의수준 (α): {alpha}")
    
    control_data = df[df['campaign_name'] == 'control']
    test_data = df[df['campaign_name'] == 'test']
    
    results = {}
    
    # 1. 구매 전환율 검정 (주요 지표)
    print("\n🎯 주요 지표: 구매 전환율")
    control_conversions = control_data['purchase'].sum()
    control_clicks = control_data['click'].sum()
    test_conversions = test_data['purchase'].sum()
    test_clicks = test_data['click'].sum()
    
    # 전환율에 대한 이중 비율 z-검정
    count = np.array([control_conversions, test_conversions])
    nobs = np.array([control_clicks, test_clicks])
    
    z_stat, p_value = proportions_ztest(count, nobs)
    
    # 신뢰구간
    control_ci = proportion_confint(control_conversions, control_clicks, alpha=alpha)
    test_ci = proportion_confint(test_conversions, test_clicks, alpha=alpha)
    
    control_rate = control_conversions / control_clicks
    test_rate = test_conversions / test_clicks
    lift = (test_rate - control_rate) / control_rate * 100
    
    results['conversion_rate'] = {
        'control_rate': control_rate,
        'test_rate': test_rate,
        'lift_percent': lift,
        'z_statistic': z_stat,
        'p_value': p_value,
        'significant': p_value < alpha,
        'control_ci': control_ci,
        'test_ci': test_ci
    }
    
    print(f"   컨트롤 전환율: {control_rate:.4f} ({control_rate*100:.2f}%)")
    print(f"   테스트 전환율: {test_rate:.4f} ({test_rate*100:.2f}%)")
    print(f"   증감률: {lift:+.2f}%")
    print(f"   Z-통계량: {z_stat:.4f}")
    print(f"   P-값: {p_value:.6f}")
    print(f"   통계적 유의성: {'✅ 예' if p_value < alpha else '❌ 아니오'} (α = {alpha})")
    
    # 2. 구매당 비용(CPA) 검정
    print("\n💰 보조 지표: 구매당 비용")
    control_cpa = control_data['spend'].sum() / control_conversions
    test_cpa = test_data['spend'].sum() / test_conversions
    
    # 통계 검정을 위한 일별 CPA
    control_daily_cpa = (control_data['spend'] / control_data['purchase']).replace([np.inf, -np.inf], np.nan).dropna()
    test_daily_cpa = (test_data['spend'] / test_data['purchase']).replace([np.inf, -np.inf], np.nan).dropna()
    
    if len(control_daily_cpa) > 1 and len(test_daily_cpa) > 1:
        # 정규성 검정
        control_normal = stats.shapiro(control_daily_cpa)[1] > 0.05
        test_normal = stats.shapiro(test_daily_cpa)[1] > 0.05
        
        if control_normal and test_normal:
            stat, p_val = stats.ttest_ind(control_daily_cpa, test_daily_cpa)
            test_type = "독립 t-검정"
        else:
            stat, p_val = stats.mannwhitneyu(control_daily_cpa, test_daily_cpa)
            test_type = "Mann-Whitney U 검정"
        
        cpa_lift = (test_cpa - control_cpa) / control_cpa * 100
        
        results['cpa'] = {
            'control_cpa': control_cpa,
            'test_cpa': test_cpa,
            'lift_percent': cpa_lift,
            'test_type': test_type,
            'statistic': stat,
            'p_value': p_val,
            'significant': p_val < alpha
        }
        
        print(f"   컨트롤 CPA: ${control_cpa:.2f}")
        print(f"   테스트 CPA: ${test_cpa:.2f}")
        print(f"   변화율: {cpa_lift:+.2f}%")
        print(f"   사용된 검정: {test_type}")
        print(f"   P-값: {p_val:.6f}")
        print(f"   통계적 유의성: {'✅ 예' if p_val < alpha else '❌ 아니오'}")
    
    # 3. 클릭률 검정
    print("\n👆 참여 지표: 클릭률")
    control_impressions = control_data['impression'].sum()
    test_impressions = test_data['impression'].sum()
    
    ctr_count = np.array([control_clicks, test_clicks])
    ctr_nobs = np.array([control_impressions, test_impressions])
    
    ctr_z_stat, ctr_p_value = proportions_ztest(ctr_count, ctr_nobs)
    
    control_ctr = control_clicks / control_impressions
    test_ctr = test_clicks / test_impressions
    ctr_lift = (test_ctr - control_ctr) / control_ctr * 100
    
    results['ctr'] = {
        'control_ctr': control_ctr,
        'test_ctr': test_ctr,
        'lift_percent': ctr_lift,
        'z_statistic': ctr_z_stat,
        'p_value': ctr_p_value,
        'significant': ctr_p_value < alpha
    }
    
    print(f"   컨트롤 CTR: {control_ctr:.4f} ({control_ctr*100:.2f}%)")
    print(f"   테스트 CTR: {test_ctr:.4f} ({test_ctr*100:.2f}%)")
    print(f"   증감률: {ctr_lift:+.2f}%")
    print(f"   P-값: {ctr_p_value:.6f}")
    print(f"   통계적 유의성: {'✅ 예' if ctr_p_value < alpha else '❌ 아니오'}")
    
    return results

# 통계 검정 수행
test_results = statistical_testing_suite(primary_df)

## 5. 퍼널 분석 및 전환 경로 최적화

In [ ]:
def funnel_analysis(df):
    """
    단계별 전환율이 포함된 포괄적인 퍼널 분석
    """
    print("\n" + "="*60)
    print("           🔄 캠페인 퍼널 분석")
    print("="*60)
    
    # 캠페인별 퍼널 지표 집계
    funnel_data = df.groupby('campaign_name').agg({
        'impression': 'sum',
        'click': 'sum', 
        'search': 'sum',
        'view': 'sum',
        'cart': 'sum',
        'purchase': 'sum'
    }).reset_index()
    
    # 각 단계별 전환율 계산
    def calculate_funnel_rates(row):
        rates = {
            'campaign': row['campaign_name'],
            'impressions': int(row['impression']),
            'clicks': int(row['click']),
            'searches': int(row['search']),
            'views': int(row['view']),
            'carts': int(row['cart']),
            'purchases': int(row['purchase']),
            
            # 전환율
            'impression_to_click': row['click'] / row['impression'],
            'click_to_search': row['search'] / row['click'],
            'search_to_view': row['view'] / row['search'],
            'view_to_cart': row['cart'] / row['view'],
            'cart_to_purchase': row['purchase'] / row['cart'],
            
            # 전체 전환율
            'overall_conversion': row['purchase'] / row['impression']
        }
        return pd.Series(rates)
    
    funnel_rates = funnel_data.apply(calculate_funnel_rates, axis=1)
    
    print("\n📊 퍼널 전환율:")
    print("\n" + "-"*80)
    
    for _, row in funnel_rates.iterrows():
        campaign = row['campaign'].upper()
        print(f"\n🎯 {campaign} 캠페인:")
        print(f"   노출 → 클릭:        {row['impression_to_click']*100:6.2f}%  ({row['impressions']:,} → {row['clicks']:,})")
        print(f"   클릭 → 검색:        {row['click_to_search']*100:6.2f}%  ({row['clicks']:,} → {row['searches']:,})")
        print(f"   검색 → 조회:        {row['search_to_view']*100:6.2f}%  ({row['searches']:,} → {row['views']:,})")
        print(f"   조회 → 장바구니:    {row['view_to_cart']*100:6.2f}%  ({row['views']:,} → {row['carts']:,})")
        print(f"   장바구니 → 구매:    {row['cart_to_purchase']*100:6.2f}%  ({row['carts']:,} → {row['purchases']:,})")
        print(f"   " + "-"*50)
        print(f"   전체 전환:          {row['overall_conversion']*100:6.4f}%  ({row['impressions']:,} → {row['purchases']:,})")
    
    # 각 퍼널 단계별 증감률 계산
    control_row = funnel_rates[funnel_rates['campaign'] == 'control'].iloc[0]
    test_row = funnel_rates[funnel_rates['campaign'] == 'test'].iloc[0]
    
    print("\n🚀 퍼널 단계별 성과 비교:")
    print("\n" + "-"*80)
    
    funnel_steps = [
        ('노출 → 클릭', 'impression_to_click'),
        ('클릭 → 검색', 'click_to_search'),
        ('검색 → 조회', 'search_to_view'),
        ('조회 → 장바구니', 'view_to_cart'),
        ('장바구니 → 구매', 'cart_to_purchase'),
        ('전체 전환', 'overall_conversion')
    ]
    
    funnel_comparison = []
    
    for step_name, metric in funnel_steps:
        control_rate = control_row[metric]
        test_rate = test_row[metric]
        lift = (test_rate - control_rate) / control_rate * 100 if control_rate > 0 else 0
        
        better = '🟢 테스트' if test_rate > control_rate else '🔴 컨트롤'
        
        funnel_comparison.append({
            'step': step_name,
            'control_rate': control_rate,
            'test_rate': test_rate,
            'lift': lift,
            'better_performer': better
        })
        
        print(f"{step_name:15} | 컨트롤: {control_rate*100:6.2f}% | 테스트: {test_rate*100:6.2f}% | 증감률: {lift:+6.1f}% | {better}")
    
    return funnel_rates, pd.DataFrame(funnel_comparison)

# 퍼널 분석 수행
funnel_rates, funnel_comparison = funnel_analysis(primary_df)

## 6. 고급 시각화

In [ ]:
def create_comprehensive_visualizations(df, funnel_rates, test_results):
    """
    A/B 테스트 분석을 위한 포괄적인 시각화 생성
    """
    
    # 1. 퍼널 차트
    fig_funnel = make_subplots(
        rows=1, cols=2,
        subplot_titles=('컨트롤 캠페인 퍼널', '테스트 캠페인 퍼널'),
        specs=[[{"type": "funnel"}, {"type": "funnel"}]]
    )
    
    # 컨트롤 퍼널
    control_data = funnel_rates[funnel_rates['campaign'] == 'control'].iloc[0]
    fig_funnel.add_trace(go.Funnel(
        y=["노출", "클릭", "검색", "조회", "장바구니", "구매"],
        x=[control_data['impressions'], control_data['clicks'], control_data['searches'], 
           control_data['views'], control_data['carts'], control_data['purchases']],
        name="컨트롤",
        marker_color="lightblue"
    ), row=1, col=1)
    
    # 테스트 퍼널
    test_data = funnel_rates[funnel_rates['campaign'] == 'test'].iloc[0]
    fig_funnel.add_trace(go.Funnel(
        y=["노출", "클릭", "검색", "조회", "장바구니", "구매"],
        x=[test_data['impressions'], test_data['clicks'], test_data['searches'], 
           test_data['views'], test_data['carts'], test_data['purchases']],
        name="테스트",
        marker_color="lightcoral"
    ), row=1, col=2)
    
    fig_funnel.update_layout(title_text="캠페인 퍼널 비교", height=500)
    fig_funnel.show()
    
    # 2. 주요 지표 비교
    fig_metrics = make_subplots(
        rows=2, cols=2,
        subplot_titles=('전환율', '구매당 비용', '클릭률', '일별 성과'),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )
    
    # 전환율
    fig_metrics.add_trace(go.Bar(
        x=['컨트롤', '테스트'],
        y=[test_results['conversion_rate']['control_rate']*100, 
           test_results['conversion_rate']['test_rate']*100],
        name='전환율 (%)',
        marker_color=['lightblue', 'lightcoral']
    ), row=1, col=1)
    
    # CPA (가능한 경우)
    if 'cpa' in test_results:
        fig_metrics.add_trace(go.Bar(
            x=['컨트롤', '테스트'],
            y=[test_results['cpa']['control_cpa'], test_results['cpa']['test_cpa']],
            name='구매당 비용 ($)',
            marker_color=['lightblue', 'lightcoral']
        ), row=1, col=2)
    
    # CTR
    fig_metrics.add_trace(go.Bar(
        x=['컨트롤', '테스트'],
        y=[test_results['ctr']['control_ctr']*100, test_results['ctr']['test_ctr']*100],
        name='클릭률 (%)',
        marker_color=['lightblue', 'lightcoral']
    ), row=2, col=1)
    
    # 시간별 일별 전환율
    daily_conv = df.copy()
    daily_conv['conversion_rate'] = daily_conv['purchase'] / daily_conv['click'] * 100
    daily_conv = daily_conv.replace([np.inf, -np.inf], np.nan).dropna()
    
    for campaign in ['control', 'test']:
        campaign_data = daily_conv[daily_conv['campaign_name'] == campaign]
        campaign_name = '컨트롤' if campaign == 'control' else '테스트'
        fig_metrics.add_trace(go.Scatter(
            x=campaign_data['date'],
            y=campaign_data['conversion_rate'],
            mode='lines+markers',
            name=f'{campaign_name} 일별 전환율',
            line=dict(color='lightblue' if campaign == 'control' else 'lightcoral')
        ), row=2, col=2)
    
    fig_metrics.update_layout(title_text="주요 성과 지표 비교", height=800, showlegend=True)
    fig_metrics.show()
    
    # 3. 통계적 유의성 요약
    fig_significance = go.Figure()
    
    metrics = ['전환율', '클릭률']
    if 'cpa' in test_results:
        metrics.append('구매당 비용')
    
    p_values = [test_results['conversion_rate']['p_value'], 
               test_results['ctr']['p_value']]
    if 'cpa' in test_results:
        p_values.append(test_results['cpa']['p_value'])
    
    colors = ['green' if p < 0.05 else 'red' for p in p_values]
    
    fig_significance.add_trace(go.Bar(
        x=metrics,
        y=[-np.log10(p) for p in p_values],
        marker_color=colors,
        name='통계적 유의성'
    ))
    
    fig_significance.add_hline(y=-np.log10(0.05), line_dash="dash", line_color="black", 
                              annotation_text="α = 0.05")
    
    fig_significance.update_layout(
        title="통계적 유의성 검정 결과 (-log10 p-value)",
        yaxis_title="-log10(p-value)",
        xaxis_title="지표",
        height=400
    )
    fig_significance.show()

# 시각화 생성
create_comprehensive_visualizations(primary_df, funnel_rates, test_results)

## 7. 시나리오별 민감도 분석

In [ ]:
def sensitivity_analysis(scenarios):
    """
    다양한 데이터 처리 시나리오에 걸친 결론의 견고성 테스트
    """
    print("\n" + "="*60)
    print("           🔬 민감도 분석")
    print("="*60)
    print("다양한 결측 데이터 처리 방법에 따른 결론의 견고성 테스트\n")
    
    sensitivity_results = {}
    
    for scenario_name, df in scenarios.items():
        print(f"\n📊 시나리오: {scenario_name.replace('_', ' ').title()}")
        print("-" * 50)
        
        # 각 시나리오별 간단한 전환율 검정
        control_data = df[df['campaign_name'] == 'control']
        test_data = df[df['campaign_name'] == 'test']
        
        control_conversions = control_data['purchase'].sum()
        control_clicks = control_data['click'].sum()
        test_conversions = test_data['purchase'].sum()
        test_clicks = test_data['click'].sum()
        
        if control_clicks > 0 and test_clicks > 0:
            control_rate = control_conversions / control_clicks
            test_rate = test_conversions / test_clicks
            lift = (test_rate - control_rate) / control_rate * 100
            
            # 통계 검정
            count = np.array([control_conversions, test_conversions])
            nobs = np.array([control_clicks, test_clicks])
            z_stat, p_value = proportions_ztest(count, nobs)
            
            sensitivity_results[scenario_name] = {
                'control_rate': control_rate,
                'test_rate': test_rate,
                'lift': lift,
                'p_value': p_value,
                'significant': p_value < 0.05
            }
            
            print(f"컨트롤 전환율: {control_rate*100:.2f}%")
            print(f"테스트 전환율: {test_rate*100:.2f}%")
            print(f"증감률: {lift:+.1f}%")
            print(f"P-값: {p_value:.6f}")
            print(f"통계적 유의성: {'✅ 예' if p_value < 0.05 else '❌ 아니오'}")
    
    # 민감도 분석 요약
    print("\n" + "="*60)
    print("           📋 민감도 요약")
    print("="*60)
    
    significant_count = sum(1 for result in sensitivity_results.values() if result['significant'])
    total_scenarios = len(sensitivity_results)
    
    print(f"\n🎯 유의한 결과를 보인 시나리오: {significant_count}/{total_scenarios}")
    
    if significant_count == total_scenarios:
        print("✅ 견고함: 모든 시나리오에서 유의한 결과 - 결론이 신뢰할 만함")
    elif significant_count > total_scenarios/2:
        print("⚠️  보통: 대부분 시나리오에서 유의한 결과 - 결론이 적당히 신뢰할 만함")
    else:
        print("❌ 민감함: 결과가 결측 데이터 처리에 민감 - 주의가 필요함")
    
    # 증감률 범위 표시
    lifts = [result['lift'] for result in sensitivity_results.values()]
    print(f"\n📈 증감률 범위: {min(lifts):+.1f}%에서 {max(lifts):+.1f}%")
    print(f"📊 평균 증감률: {np.mean(lifts):+.1f}% (±{np.std(lifts):.1f}%)")
    
    return sensitivity_results

# 민감도 분석 수행
sensitivity_results = sensitivity_analysis(analysis_scenarios)

## 8. 비즈니스 영향 및 권장사항

In [ ]:
def generate_business_recommendations(test_results, funnel_comparison, sensitivity_results):
    """
    통계 분석을 바탕으로 한 포괄적인 비즈니스 권장사항 생성
    """
    print("\n" + "="*80)
    print("                    📈 비즈니스 영향 및 권장사항")
    print("="*80)
    
    # 주요 지표 분석
    conversion_result = test_results['conversion_rate']
    
    print("\n🎯 주요 발견사항: 구매 전환율")
    print("-" * 60)
    
    if conversion_result['significant']:
        winner = '테스트' if conversion_result['test_rate'] > conversion_result['control_rate'] else '컨트롤'
        print(f"✅ 통계적으로 유의한 결과 발견")
        print(f"🏆 승자: {winner} 캠페인")
        print(f"📊 전환율 개선: {abs(conversion_result['lift_percent']):.1f}%")
        print(f"📈 {conversion_result['control_rate']*100:.2f}%에서 {conversion_result['test_rate']*100:.2f}%로")
        
        # 비즈니스 영향 추정
        if winner == '테스트':
            print(f"\n💰 비즈니스 영향 예측:")
            print(f"   • 클릭 1000회당 약 {(conversion_result['test_rate'] - conversion_result['control_rate'])*1000:.0f}건의 추가 구매 예상")
            print(f"   • 전환 성과에서 {conversion_result['lift_percent']:+.1f}%의 상대적 개선")
            
            recommendation = "권장사항: 테스트 캠페인 구현"
            action = "🚀 즉시 실행: 모든 마케팅 활동에 테스트 캠페인 방식을 확산"
        else:
            recommendation = "권장사항: 컨트롤 캠페인 지속"
            action = "⚠️  변경 전 테스트 캠페인의 저조한 성과 원인 조사 필요"
            
    else:
        print(f"❌ 통계적으로 유의한 차이 없음")
        print(f"📊 관찰된 차이: {conversion_result['lift_percent']:+.1f}%")
        print(f"🔬 P-값: {conversion_result['p_value']:.6f} (α=0.05에서 유의하지 않음)")
        
        recommendation = "권장사항: 변경 불필요"
        action = "🔄 현재 방식 유지 또는 더 높은 검정력을 위해 더 긴 테스트 실행"
    
    print(f"\n🎯 {recommendation}")
    print(f"📋 {action}")
    
    # 퍼널 인사이트
    print("\n\n🔄 퍼널 최적화 기회")
    print("-" * 60)
    
    # 퍼널에서 가장 큰 증가와 가장 큰 감소 찾기
    funnel_lifts = funnel_comparison.set_index('step')['lift']
    best_step = funnel_lifts.idxmax()
    worst_step = funnel_lifts.idxmin()
    
    print(f"🟢 최고 성과 단계: {best_step}")
    print(f"   • 테스트 캠페인에서 {funnel_lifts[best_step]:+.1f}% 개선")
    print(f"   • 권장사항: 이 성공 사례를 연구하고 다른 캠페인에 적용")
    
    print(f"\n🔴 저조한 성과 단계: {worst_step}")
    print(f"   • 테스트 캠페인에서 {funnel_lifts[worst_step]:+.1f}% 변화")
    if funnel_lifts[worst_step] < 0:
        print(f"   • 권장사항: 테스트 캠페인에서 이 단계가 저조한 이유 조사")
    else:
        print(f"   • 이 단계도 테스트 캠페인에서 개선됨")
    
    # 민감도 분석 해석
    print("\n\n🔬 분석 신뢰성")
    print("-" * 60)
    
    significant_scenarios = sum(1 for r in sensitivity_results.values() if r['significant'])
    total_scenarios = len(sensitivity_results)
    
    if significant_scenarios == total_scenarios:
        reliability = "높음"
        confidence = "권장사항에 매우 높은 신뢰"
    elif significant_scenarios > total_scenarios/2:
        reliability = "보통"
        confidence = "권장사항에 적당한 신뢰"
    else:
        reliability = "낮음"
        confidence = "낮은 신뢰도 - 더 많은 데이터 필요"
    
    print(f"📊 분석 신뢰성: {reliability}")
    print(f"🎯 신뢰 수준: {confidence}")
    print(f"✅ {total_scenarios}개 데이터 시나리오 중 {significant_scenarios}개에서 일관된 결과")
    
    # 다음 단계
    print("\n\n📋 권장 다음 단계")
    print("-" * 60)
    
    if conversion_result['significant'] and reliability == "높음":
        print("1. 🚀 즉시: 승리한 캠페인 전략 구현")
        print("2. 📈 모니터링: 2-4주간 성과 지표 면밀히 추적")
        print("3. 🔍 분석: 승리한 캠페인을 성공적으로 만든 요인 심층 분석")
        print("4. 📊 최적화: 학습 내용을 다른 마케팅 캠페인 개선에 적용")
    elif not conversion_result['significant']:
        print("1. 🔄 연장: 통계적 검정력 증가를 위해 테스트 기간 연장")
        print("2. 📊 증가: 향후 테스트에서 더 큰 표본 크기 고려")
        print("3. 🎯 집중: 캠페인 간 더 극명한 차이 테스트")
        print("4. 📈 모니터링: 더 명확한 차이를 보일 수 있는 보조 지표 추적")
    else:
        print("1. 🔍 조사: 신뢰성에 영향을 미치는 데이터 품질 문제 파악")
        print("2. 🧹 개선: 데이터 수집 프로세스 향상")
        print("3. 🔄 반복: 더 나은 데이터 품질 통제로 테스트 재실행")
        print("4. 📋 검증: 추가 테스트로 결과 확인")
    
    print("\n" + "="*80)
    print("                           ✅ 분석 완료")
    print("="*80)

# 포괄적인 비즈니스 권장사항 생성
generate_business_recommendations(test_results, funnel_comparison, sensitivity_results)

## 9. 통계 요약 보고서

In [ ]:
def generate_executive_summary(test_results, funnel_rates):
    """
    이해관계자를 위한 경영진 요약 보고서 생성
    """
    print("\n" + "="*80)
    print("                        📊 경영진 요약표")
    print("="*80)
    
    # 요약 데이터프레임 생성
    summary_data = []
    
    # 전환율
    cr = test_results['conversion_rate']
    summary_data.append({
        '지표': '구매 전환율',
        '컨트롤': f"{cr['control_rate']*100:.2f}%",
        '테스트': f"{cr['test_rate']*100:.2f}%",
        '증감률': f"{cr['lift_percent']:+.1f}%",
        'P-값': f"{cr['p_value']:.4f}",
        '유의성': '✅' if cr['significant'] else '❌',
        '승자': '테스트' if cr['test_rate'] > cr['control_rate'] else '컨트롤'
    })
    
    # 클릭률
    ctr = test_results['ctr']
    summary_data.append({
        '지표': '클릭률',
        '컨트롤': f"{ctr['control_ctr']*100:.2f}%",
        '테스트': f"{ctr['test_ctr']*100:.2f}%",
        '증감률': f"{ctr['lift_percent']:+.1f}%",
        'P-값': f"{ctr['p_value']:.4f}",
        '유의성': '✅' if ctr['significant'] else '❌',
        '승자': '테스트' if ctr['test_ctr'] > ctr['control_ctr'] else '컨트롤'
    })
    
    # CPA (가능한 경우)
    if 'cpa' in test_results:
        cpa = test_results['cpa']
        summary_data.append({
            '지표': '구매당 비용',
            '컨트롤': f"${cpa['control_cpa']:.2f}",
            '테스트': f"${cpa['test_cpa']:.2f}",
            '증감률': f"{cpa['lift_percent']:+.1f}%",
            'P-값': f"{cpa['p_value']:.4f}",
            '유의성': '✅' if cpa['significant'] else '❌',
            '승자': '테스트' if cpa['test_cpa'] < cpa['control_cpa'] else '컨트롤'  # CPA는 낮을수록 좋음
        })
    
    # 전체 퍼널 성과
    control_overall = funnel_rates[funnel_rates['campaign'] == 'control']['overall_conversion'].iloc[0]
    test_overall = funnel_rates[funnel_rates['campaign'] == 'test']['overall_conversion'].iloc[0]
    overall_lift = (test_overall - control_overall) / control_overall * 100
    
    summary_data.append({
        '지표': '전체 퍼널 전환',
        '컨트롤': f"{control_overall*100:.3f}%",
        '테스트': f"{test_overall*100:.3f}%",
        '증감률': f"{overall_lift:+.1f}%",
        'P-값': '주요 지표 참조',
        '유의성': '✅' if cr['significant'] else '❌',
        '승자': '테스트' if test_overall > control_overall else '컨트롤'
    })
    
    # 요약표 생성 및 표시
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    
    # 핵심 요약
    print("\n\n🔑 핵심 요약:")
    print("-" * 50)
    
    if cr['significant']:
        winner = '테스트' if cr['test_rate'] > cr['control_rate'] else '컨트롤'
        print(f"• {winner} 캠페인이 통계적으로 유의한 개선을 보임")
        print(f"• 주요 전환율이 {abs(cr['lift_percent']):.1f}% 개선")
        
        if winner == '테스트':
            print(f"• 테스트 캠페인 전략 구현 권장")
        else:
            print(f"• 컨트롤 캠페인 유지 권장")
    else:
        print(f"• 캠페인 간 통계적으로 유의한 차이 없음")
        print(f"• 더 긴 테스트 기간이나 더 큰 표본 크기 고려")
        print(f"• 현재 접근 방식이 적절해 보임")
    
    return summary_df

# 경영진 요약 생성
executive_summary = generate_executive_summary(test_results, funnel_rates)

## 10. 결론

### 📋 분석 방법론 요약:
1. **데이터 품질 평가**: 데이터 완전성 및 무결성에 대한 포괄적 평가
2. **민감도 분석**: 여러 결측 데이터 처리 시나리오에 걸친 견고성 테스트
3. **통계 검정**: 적절한 통계 방법을 사용한 엄격한 가설 검정
4. **퍼널 분석**: 최적화 기회를 식별하기 위한 단계별 전환 분석
5. **비즈니스 영향**: 통계 결과를 실행 가능한 비즈니스 권장사항으로 변환

### 🎯 통계적 엄밀성:
- 전환율 비교를 위한 이중 비율 z-검정
- 효과 크기 추정을 위한 신뢰구간
- 결과 검증을 위한 다중 시나리오 테스트
- 데이터 특성에 기반한 적절한 통계 검정 선택

### 📈 비즈니스 가치:
이 분석은 통계적 엄밀성을 보장하면서 실용적인 비즈니스 결과에 집중하여 데이터 기반 캠페인 최적화 결정을 위한 완전한 프레임워크를 제공합니다.